In [1]:
import os

import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from lasagne import init
from lasagne.layers import (Layer, InputLayer, MergeLayer, DenseLayer, 
                            DimshuffleLayer, ElemwiseSumLayer, 
                            ReshapeLayer, NonlinearityLayer, 
                            get_all_params, get_output)
from lasagne.objectives import categorical_crossentropy
from lasagne.nonlinearities import rectify, sigmoid, softmax
from lasagne.updates import total_norm_constraint, adam
from parmesan.layers import SampleLayer
from parmesan.distributions import log_normal
from models.adgmssl import ADGMSSL


DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmp8pjzjr/265abc51f7c376c224983485238ff1a5.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmp8pjzjr/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, CuDNN 4004)
d:\packages\theano\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


EVIL HACK: Disable cuDNN check


In [2]:
from data_preparation import mnist
import numpy as np

In [3]:
"""
Train a auxiliary deep generative model on the mnist dataset with 100 evenly distributed labels.
"""
n_labeled = 100  # The total number of labeled data points.
n_samples = 100  # The number of sampled labeled data points for each batch.
n_batches = 100  # The number of batches.
mnist_data = mnist.load_semi_supervised(n_batches=n_batches, n_labeled=n_labeled, n_samples=n_samples,
                                        filter_std=0.0, seed=123456, train_valid_combine=True)

# mnist_data[0][0] = mnist_data[0][0].reshape((-1, 1, 28, 28))
# mnist_data[1][0] = mnist_data[1][0].reshape((-1, 1, 28, 28))
# mnist_data[2][0] = mnist_data[2][0].reshape((-1, 1, 28, 28))
# shape_x = (1,28,28)
shape_x = (28*28,)
n = mnist_data[0][0].shape[0]  # Datapoints in the dataset, input features.
bs = n / n_batches  # The batchsize.

def f_enc(layer):
    return DenseLayer(layer, 500, init.GlorotNormal('relu'), init.Normal(1e-3), rectify)

def f_dec(layer):
    return DenseLayer(layer, 500, init.GlorotNormal('relu'), init.Normal(1e-3), rectify)

# Initialize the auxiliary deep generative model.
model = ADGMSSL(shape_x=shape_x, n_a=100, n_z=100, n_y=10, n_xhat=500, f_enc=f_enc, f_dec=f_dec,
                z_hidden=[500], y_hidden=[500],
                trans_func=rectify, x_dist='bernoulli')

# Get the training functions.
f_train, f_test, f_validate, train_args, test_args, validate_args = model.build_model(*mnist_data)

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpqfgq_v/537b91f68a183917beba9222d778e10a.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpqfgq_v/537b91f68a183917beba9222d778e10a.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmphpohpa/4bac00df9fcbfcd9da45c4e28a1178fa.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmphpohpa/4bac00df9fcbfcd9da45c4e28a1178fa.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpcz5axo/3092c38da93a783358837b10117af3a4.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Inte

### BUILDING MODEL ###


 mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpd9yheh/2fddde1b8013896b3d970643f492261c.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpd9yheh/2fddde1b8013896b3d970643f492261c.exp



In [4]:
import time
import cPickle as pkl
import os
path = "output/notebook"
if not os.path.exists(path):
    os.mkdir(path)

bs_l = n_samples
beta = 1200.
lr = 3e-4
beta1 = 0.9
beta2 = 0.999
samples = 1
epochs = 200
for epoch in range(epochs):
    train_outputs = []
    start = time.time()
    for i in xrange(n_batches):
        train_output = f_train(i, bs, bs_l, beta, lr, beta1, beta2, samples)
        train_outputs.append(train_output)
    lb = np.mean(np.array(train_outputs), axis=0)
    end = time.time() - start
    
    test_outputs = f_test(samples)
    class_err_test = np.mean(np.array(test_outputs), axis=0)
    
    # model_params = [param.get_value() for param in model]
    # pkl.dump(model_params, open(os.path.join(path, "model.pkl"), "wb"))
    
    print "[epoch,time,train[lb],test[err]];%i;%0.2f;%0.4f;%0.2f%%" % (epoch+1, end, lb, class_err_test)

[epoch,time,train[lb],test[err]];1;26.20;617.7183;75.79%
[epoch,time,train[lb],test[err]];2;26.25;527.8387;68.90%
[epoch,time,train[lb],test[err]];3;26.55;512.1379;66.09%
[epoch,time,train[lb],test[err]];4;26.15;494.6202;63.05%
[epoch,time,train[lb],test[err]];5;25.97;476.6184;59.52%
[epoch,time,train[lb],test[err]];6;26.27;453.5691;57.05%
[epoch,time,train[lb],test[err]];7;26.15;421.8041;51.48%
[epoch,time,train[lb],test[err]];8;25.93;389.7505;46.04%
[epoch,time,train[lb],test[err]];9;26.18;357.2982;40.84%
[epoch,time,train[lb],test[err]];10;26.01;327.0761;37.23%
[epoch,time,train[lb],test[err]];11;26.01;307.1738;36.01%
[epoch,time,train[lb],test[err]];12;26.06;291.0773;34.76%
[epoch,time,train[lb],test[err]];13;26.01;270.8420;32.14%
[epoch,time,train[lb],test[err]];14;26.13;253.9373;30.99%
[epoch,time,train[lb],test[err]];15;26.07;238.5486;29.27%
[epoch,time,train[lb],test[err]];16;26.23;221.9402;27.63%
[epoch,time,train[lb],test[err]];17;26.17;209.4787;26.29%
[epoch,time,train[lb],t